In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [167]:
def snake_str(x: str):
    x = x.replace(' ', '_')
    return x.lower()


customer_df = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')
customer_df.columns = list(map(snake_str, customer_df.columns)) 
customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])
customer_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2011-02-10,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2011-02-12,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2011-02-06,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2011-02-03,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [168]:
customer_df['policy'] = customer_df['policy'].apply(lambda x: str.split(x, ' ')[1])
customer_df['policy_type'] = customer_df['policy_type'].apply(lambda x: str.split(x, ' ')[0])

In [169]:
y = customer_df['total_claim_amount'].copy()
X = customer_df.drop(['total_claim_amount'], axis=1).copy()
X.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,...,32,5,0,1,Corporate,L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,...,13,42,0,8,Personal,L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,...,18,38,0,2,Personal,L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,...,18,65,0,7,Corporate,L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,...,12,44,0,1,Personal,L1,Offer1,Agent,Four-Door Car,Medsize


### the ordinal encoding makes the models about 10% less accurate !!! 

#### so i rather use onehot there too

In [170]:
# from sklearn.preprocessing import OrdinalEncoder

# ordin = OrdinalEncoder()
# X['coverage'] = ordin.fit_transform(X[['coverage']])
# X['employmentstatus'] = ordin.fit_transform(X[['employmentstatus']])
# X['location_code'] = ordin.fit_transform(X[['location_code']])
# X['vehicle_size'] = ordin.fit_transform(X[['vehicle_size']])

In [171]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [172]:
def get_outliers(column):
    # find q1 and q3 values and calculate iqr
    q1, q3 = np.percentile(sorted(column), [25, 75])
    iqr = q3 - q1
 
    # lower and upper limit
    lower_limit = q1 - (1.5 * iqr)
    upper_limit = q3 + (1.5 * iqr)
 
    outliers = [x for x in column if x <= lower_limit or x >= upper_limit]
    return outliers

In [173]:
# from lab num-cleaning, columns with outliers: monthly_premium_auto, total_claim_amount, customer_lifetime_value

mpa_outliers = get_outliers(customer_df['monthly_premium_auto'])
tca_outliers = get_outliers(customer_df['total_claim_amount'])
clv_outliers = get_outliers(customer_df['customer_lifetime_value'])
print(len(customer_df))
customer_df = customer_df[~customer_df['monthly_premium_auto'].isin(mpa_outliers)].copy()
customer_df = customer_df[~customer_df['total_claim_amount'].isin(tca_outliers)].copy()
customer_df = customer_df[~customer_df['customer_lifetime_value'].isin(clv_outliers)].copy()
print(len(customer_df))

9134
7847


In [174]:
X_train_nums = X_train.select_dtypes(include=np.number)
X_train_nums = X_train_nums.drop(['months_since_last_claim',
 'months_since_policy_inception',
 'number_of_open_complaints',
 'number_of_policies'],axis=1)
X_train_nums

,customer_lifetime_value,income,monthly_premium_auto
3238,25997.749960,62262,72
1297,4330.386020,60475,107
4576,7550.418650,0,100
2497,3911.118202,30821,97
3656,5568.947534,0,79
...,...,...,...
329,11264.363340,34923,98
2459,4686.469674,58089,118
1151,3600.934326,25147,101
4025,4103.849192,82427,102


In [175]:
X_test_nums = X_test.select_dtypes(include=np.number)
X_test_nums = X_test_nums.drop(['months_since_last_claim',
 'months_since_policy_inception',
 'number_of_open_complaints',
 'number_of_policies'],axis=1)
X_test_nums

,customer_lifetime_value,income,monthly_premium_auto
8223,17817.464640,0,68
3380,5681.307056,67132,71
4501,2462.859762,0,69
7595,7255.137772,67352,181
5643,2424.424678,27093,61
...,...,...,...
4417,15956.518790,61148,132
8919,2944.780529,87274,73
7176,2725.322949,45025,69
5625,15114.402410,28513,100


In [176]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train_nums)
X_train_norm = scaler.transform(X_train_nums)
X_train_norm = pd.DataFrame(X_train_norm,columns=X_train_nums.columns)

X_test_norm = scaler.transform(X_test_nums)
X_test_norm = pd.DataFrame(X_test_norm,columns=X_train_nums.columns)

In [177]:
X_num_cats = X_train[['months_since_last_claim',
 'months_since_policy_inception',
 'number_of_open_complaints',
 'number_of_policies']].copy()
orig_cats = X_train.select_dtypes(include=object).drop(['customer'], axis=1)
X_train_cats = pd.concat((X_num_cats, orig_cats), axis=1)

In [178]:
X_num_cats_test = X_test[['months_since_last_claim',
 'months_since_policy_inception',
 'number_of_open_complaints',
 'number_of_policies']].copy()
orig_cats = X_test.select_dtypes(include=object).drop(['customer'], axis=1)
X_test_cats = pd.concat((X_num_cats_test, orig_cats), axis=1)

In [179]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(pd.DataFrame(X_train_cats))

encoded = encoder.transform(pd.DataFrame(X_train_cats)).toarray()
encoded_test = encoder.transform(pd.DataFrame(X_test_cats)).toarray()

cols = encoder.get_feature_names_out(input_features=X_train_cats.columns)

onehot_encoded_cats = pd.DataFrame(encoded, columns=cols).astype(object)
onehot_encoded_cats_test = pd.DataFrame(encoded_test, columns=cols).astype(object)
onehot_encoded_cats

,months_since_last_claim_1,months_since_last_claim_2,months_since_last_claim_3,months_since_last_claim_4,months_since_last_claim_5,months_since_last_claim_6,months_since_last_claim_7,months_since_last_claim_8,months_since_last_claim_9,months_since_last_claim_10,...,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6847,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [180]:
# concat nums and cats
X_train_final = pd.concat((X_train_norm, onehot_encoded_cats), axis=1)
X_test_final = pd.concat((X_test_norm, onehot_encoded_cats_test), axis=1)
X_test_final

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim_1,months_since_last_claim_2,months_since_last_claim_3,months_since_last_claim_4,months_since_last_claim_5,months_since_last_claim_6,months_since_last_claim_7,...,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,0.195446,0.000000,0.029536,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.046392,0.671448,0.042194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.006864,0.000000,0.033755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.065722,0.673648,0.506329,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.006392,0.270981,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279,0.172590,0.611596,0.299578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2280,0.012783,0.872906,0.050633,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2281,0.010087,0.450336,0.033755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2282,0.162247,0.285184,0.164557,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [181]:
from sklearn.linear_model import LinearRegression

# build model
linreg = LinearRegression().fit(X_train_final, y_train)

# train score
predictions = linreg.predict(X_train_final)
print(linreg.score(X_train_final, y_train))

# test score
predictions_test = linreg.predict(X_test_final)
print(linreg.score(X_test_final, y_test))

0.7813097677615614
0.7637872357295791


In [182]:
def try_models(models: list):
    for model in models:
        classification = model.fit(X_train_final, y_train)

        predictions = classification.predict(X_train_final)
        print(model, "train :", classification.score(X_train_final, y_train))

        predictions_test = classification.predict(X_test_final)
        print(model, "test :", classification.score(X_test_final, y_test))

In [183]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor


try_models([LinearRegression(), KNeighborsRegressor(), MLPRegressor()])

LinearRegression() train : 0.7813097677615614
LinearRegression() test : 0.7637872357295791
KNeighborsRegressor() train : 0.6966468999792274
KNeighborsRegressor() test : 0.5348854170287795
MLPRegressor() train : 0.8128375251311556
MLPRegressor() test : 0.786809437397557


In [184]:
# i dont like the function

# you either can put just the regressors in the list, but then you cant specify different paramaters
# for different models (especially KNN lacks of specific n_neighbors to get a good result)

# if building (fitting) the models beforehand the function just becomes a 'print-the-scores'-function